In [1]:
import piplates.DAQC2plate as DAQC2
import os

In [18]:
# Read Barrier Position
DAQC2.getADC(0,0)-DAQC2.getADC(0,1)

0.0

In [5]:
# Open Barriers
DAQC2.setDAC(0,0,1) # set fast close
DAQC2.setDOUTbit(0,0) # turn on power/start barriers

In [50]:
def barrier_at_limit_check(openlimit, closelimit):
    """
    Checks if barrier is at or beyond limit and stops barrier if it is moving
    in a direction that would make it worse.
    :param float openlimit: lowest voltage allowed for opening
    :param float closelimit: highest voltage allowed for closing

    Returns
    =======
    True or False. If True also shuts down power to barrier
    """

    direction = 0  # -1 closing, 0 stopped, 1 openning.
    if (DAQC2.getDAC(0, 0) >= 2.5):
        direction = -1
    else:
        direction = 1
    position = DAQC2.getADC(0, 0) - DAQC2.getADC(0, 1)
    if (position >= closelimit) and (direction == -1):
        DAQC2.clrDOUTbit(0, 0)  # switch off power/stop barriers
        print(position,direction)
        return True
    if (position <= openlimit) and (direction == 1):
        DAQC2.clrDOUTbit(0, 0)  # switch off power/stop barriers
        print(position,direction)
        return True
    return False

In [53]:
def motorcal(barriermin,barriermax):
    '''
    :param float barriermin: minimum voltage for barrier (do not open more).
    :param float barriermax: maximum voltage for barrier (do not close more).
    :returns float maxclose: DAC setting for maximum close speed.
    :returns float minclose: DAC setting for minimum close speed.
    :returns float startclose: DAC setting providing minimum voltage to start closing.
    :returns float maxopen: DAC setting for maximum close speed.
    :returns float minopen: DAC setting for minimum close speed.
    :returns float startopen: DAC setting providing minimum voltage to start opening.
    '''
    
    import os, time
    # Since this runs in a tight loop needs to take over watching barriers.
    # Check if a trough controller is registered at /tmp/troughctl.pid.
    # If one is store it's pid and replace with own. Will revert on exit without
    # crashing.
    pidpath = '/tmp/troughctl.pid'
    ctlpid = None
    if os.path.exists(pidpath):
        file=open(pidpath,'r')
        ctlpid = int(file.readline())
        file.close()
    pid = os.getpid()
    print(str(pid))
    file = open(pidpath,'w')
    file.write(str(pid)+'\n')
    file.close()
    # Do not proceed until this file exists on the file system
    while not os.path.exists(pidpath):
        pass # just waiting for file system to catch up.
    # Read it to make sure
    file = open(pidpath,'r')
    checkpid = file.readline()
    file.close()
    if int(checkpid) != pid:
        raise FileNotFoundError('Checkpid = '+checkpid+', but should = '+str(pid))
        
    # Set base values
    maxclose = 4
    minclose = 2.6
    startclose = 2.6
    maxopen = 0
    minopen = 2.4
    startopen = 2.4
    # Calibrate the barrier. This assumes a DAQC2 pi-plate is the controlling interface.
    position = round(DAQC2.getADC(0, 0) - DAQC2.getADC(0, 1),2) # not stable past 2 decimals
    middle = (barriermax + barriermin)/2
    print('position: '+str(position)+', middle: '+str(middle))
    if position < middle: 
        # need to close some
        atlimit = False
        DAQC2.setDAC(0,0,4) # set fast close
        DAQC2.setDOUTbit(0,0) # turn on power/start barriers
        while not atlimit:
            atlimit = barrier_at_limit_check(barriermin, middle)
            time.sleep(0.2)
    if position > middle:
        # need to open some
        atlimit = False
        DAQC2.setDAC(0,0,0) # set fast open
        DAQC2.setDOUTbit(0,0) # turn on power/start barriers
        while not atlimit:
            atlimit = barrier_at_limit_check(middle, barriermax)
            time.sleep(0.2)
            
    # Return control to previous trough controller
    if ctlpid is not None:
        file = open(pidpath,'w')
        file.write(str(ctlpid)+'\n')
        file.close()
    elif os.path.exists(pidpath):
        os.remove(pidpath)
    elif os.path.exists(pidpath): # double check
        os.remove(pidpath)
    
    
    # Return the results
    return(maxclose,minclose,startclose,maxopen,minopen,startopen)
    

Some how the position measured in the barrier_at_limit_check() call is wrong. Are we looping to fast?

In [55]:
motorcal(0.05,7.78)

4061
position: 6.73, middle: 3.915
3.74 1


(4, 2.6, 2.6, 0, 2.4, 2.4)

In [46]:
DAQC2.getDAC(0, 0)

0.0